## Running two ML models at the same time
This notebook is an example how to run two models side-by-side and combine results of both models. A video stream from a local camera is processed by the hand and face detection models. Combined result is then displayed.

**Access to camera is required to run this sample.**

This script works with the following inference options:

1. [DeGirum Cloud Platform](https://cs.degirum.com),
1. DeGirum-hosted AI server node shared via Peer-to-Peer VPN,
1. AI server node hosted by you in your local network,
1. AI server running on your local machine,
1. DeGirum ORCA accelerator directly installed on your local machine.

To try different options, you just need to change the `inference_option` in the code below.

The script needs either a web camera or local camera connected to the machine running this code. The camera index or URL needs to be specified either in the code below by assigning `camera_id` or in .env file by defining `CAMERA_ID` variable and assigning `camera_id = None`.

### Specify where do you want to run your inferences and camera index here

In [ ]:
inference_option = 1  # <<< change it according to your needs selecting from the list in the header comment
camera_id = 0         # camera index or URL; 0 to use default local camera, None to take from .env file

### The rest of the cells below should run without any modifications

In [ ]:
import degirum as dg # import DeGirum PySDK
import mytools, cv2

In [ ]:
# connect to model zoo according to selected inference option
zoo = mytools.connect_model_zoo(inference_option)

In [ ]:
# load models for DeGirum Orca AI accelerator
# (change model name to "...n2x_cpu_1" to run it on CPU)
hand_det_model = zoo.load_model("yolo_v5s_hand_det--512x512_quant_n2x_orca_1")
face_det_model = zoo.load_model("yolo_v5s_face_det--512x512_quant_n2x_orca_1")

# select OpenCV backend: needed to have overlay image in OpenCV format
hand_det_model.image_backend = 'opencv'
hand_det_model._model_parameters.InputImgFmt = ['JPEG']
hand_det_model.input_numpy_colorspace = 'BGR'

face_det_model.image_backend = 'opencv' 
face_det_model._model_parameters.InputImgFmt = ['JPEG']
face_det_model.input_numpy_colorspace = 'BGR'

In [ ]:
# define iterator function, which returns frames from camera 
def source(stream, idx, bufs):
    # idx is the index of a buffer
    while True:
        if len(bufs[idx]) == 0: # this buffer is empty: get frame from camera and add to all buffers
            ret, frame = stream.read()
            if not ret:
                raise Exception("Fail to capture camera frame. May be camera was opened by another notebook?")                            
            for s in bufs:
                s.insert(0, frame)
        yield bufs[idx].pop()

In [ ]:
bufs = [[],[]]
with mytools.Display("Hands and Faces") as display, \
     mytools.open_video_stream(camera_id) as stream:
    
    # run hand and faces detection models on a camera stream side-by-side
    for hands, faces in zip(hand_det_model.predict_batch(source(stream, 0, bufs)), 
                            face_det_model.predict_batch(source(stream, 1, bufs))):
        hands._inference_results += faces._inference_results # combine inference results of two detection models
        display.show(hands.image_overlay) # show image overlay with combined results